# Python Programming Challenge

## Poker Hand

In this challenge, we have to determine which kind of Poker combination is present in a deck of 5 cards. Every card is a string containing the card value **with the upper-case initial for face-cards** and the **lower-case initial for the suit**, as seen in the examples below:

> "Ah" ➞ Ace of hearts <br>
> "Ks" ➞ King of spades<br>
> "3d" ➞ Three of diamonds<br>
> "Qc" ➞ Queen of clubs <br>

There are 10 different combinations. Here's the list, in descending order of importance:

| Name            | Description                                         |
|-----------------|-----------------------------------------------------|
| Royal Flush     | A, K, Q, J, 10, all with the same suit.             |
| Straight Flush  | Five cards in sequence, all with the same suit.     |
| Four of a Kind  | Four cards of the same rank.                        |
| Full House      | Three of a Kind with a Pair.                        |
| Flush           | Any five cards of the same suit, not in sequence    |
| Straight        | Five cards in a sequence, but not of the same suit. |
| Three of a Kind | Three cards of the same rank.                       |
| Two Pair        | Two different Pairs.                                |
| Pair            | Two cards of the same rank.                         |
| High Card       | No other valid combination.                         |

---------

#### 1. Given a list `hand` containing five strings being the cards. Implement a function called `poker_hand_ranking` that **returns a string with the name of the highest combination obtained.** According to the table above.

**Examples:**

> poker_hand_ranking(["10h", "Jh", "Qh", "Ah", "Kh"]) ➞ "Royal Flush"<br>
> poker_hand_ranking(["3h", "5h", "Qs", "9h", "Ad"]) ➞ "High Card"<br>
> poker_hand_ranking(["10s", "10c", "8d", "10d", "10h"]) ➞ "Four of a Kind"<br>

### Head

In [939]:
# import libraries
from collections import Counter
import numpy as np
import operator as op

In [940]:
# for testing
hand1 = ["10h", "Jh", "Qh", "Ah", "Kh"]
hand2 = ["3h", "5h", "Qs", "9h", "Ad"]
hand3 = ["10s", "10c", "8d", "10d", "10h"]
hand4 = ["6d", "2h", "4c", "5d", "3h"]
hand5 = ["6d", "6h", "4c", "5d", "5h"]
hand6 = ["6d", "3d", "4c", "Ad", "3h"]
hand7 = ["2d", "4c", "2h", "5d", "5h"]

### The Cards

In [941]:
# define the face conversion
face_value = [i for i in range(1,14)]
face_key = ['A'] + [str(i) for i in range(2,11)] + ['J', 'Q', 'K']
face = dict(zip(face_key, face_value))

# define the suit conversion
suit = ['h', # hearts
        's', # spades
        'd', # diamonds
        'c'] # clubs

### Functions

##### The 10-Cards

In [942]:
# evaluate if value is 10
def ten(face):
    return face[:2] == '10'

##### Card Names to Values

In [943]:
def card_values(hand):
    # if the card value is 10, use ten() to pull 2 digits rather than just the first
    values = [card[:2] if ten(card) else card[0] for card in hand]
    
    # return the hand as value pairs of value & keys, respectively
    return [face[value] for value in values], [card[-1] for card in hand]

##### Hand Configurations

In [944]:
######## FLUSHES #########

def flush(hand):
    # convert hand to values and create Counter class;
    # the Counter class counts the argument elements, then returns the results as a key/value pair
    # where the key is the element being counted
    values, suits = card_values(hand)
    suit_count = Counter(suits)
    
    # flush requires that the entire hand is the same suit
    # if the Counter returns a result of 5 for any element,
    # then we can conclude that the hand is at least a flush 
    for key, value in suit_count.items():
        if value == 5 and key in suit:
            return True
        
    return False


######## STRAIGHTS #########

def straight(hand):
    # sort values to facilitate evaluation of straight status
    values, suits = card_values(hand)
    values = sorted(values)
    
    for i, value in enumerate(values):
        # evaluate if value is 1 and continue if next value is either 2 or 10
        # a value of 1 represents an ace, which will be next to a 2 or 10
        # in a sorted list if the hand is a straight that ends or begins with an ace
        if value == 1 and (values[i+1] == 2 or values[i+1] == 10):
            continue
        elif value == 1 and not (values[i+1] == 2 or values[i+1] == 10):
            return False
        
        # evaluate if numbers increase by 1
        # if the loop reaches the end of the list, return True
        # otherwise, return False
        try:
            if value + 1 == values[i+1]:
                continue
            else:
                return False
        except IndexError:
            return True
        

######## ROYALS #########

# this is a permutation of a straight so evaluation of a royal hand is a subset
# of the straight function, which strictly begins with a hand value of 1
def royals(hand):
    # sort values to facilitate evaluation of ace in royals
    values, suits = card_values(hand)
    values = sorted(values)
    
    return straight(hand) and values == [1, 10, 11, 12, 13]
        
        
######## 4-KINDS #########

# Counter class is necessary here to reveal
# any elements that repeats 4 times
def fourkinds(hand):
    values, suits = card_values(hand)
    value_count = Counter(values)
    
    # catch if any elements repeats 4 times
    for count in value_count.values():
            if count == 4:
                return True
    
    return False


######## TTRIPLES #########

# Counter class is necessary here to reveal
# any elements that repeats 3 times
def triples(hand):
    # convert hand to values
    values, suits = card_values(hand)
    value_count = Counter(values)
    
    # catch if any elements repeats 3 times
    for count in value_count.values():
            if count == 3:
                return True
    
    return False


####### PAIRS ########

# Counter class is necessary here to reveal
# any elements that repeats twice
def pairs(hand):
    # convert hand to values
    values, suits = card_values(hand)
    value_count = Counter(values)
    
    # catch if any elements repeats twice
    for count in value_count.values():
            if count == 2:
                return True
    
    return False

##### Evaluate Hands

In [945]:
# all hand configuration evaluator functions return boolean values, True or False

def poker_hand_ranking(hand):
    
    number, suit = card_values(hand)
    
    if royals(hand) and flush(hand):
        return 'royal flush', 9, max(number)
    
    if straight(hand) and flush(hand):
        return 'straight flush', 8, max(number)
    
    if fourkinds(hand):
        return 'four of a kind', 7, max(number)
    
    if triples(hand) and pairs(hand):
        return 'full house', 6, max(number)
    
    if flush(hand):
        return 'flush', 5, max(number)
    
    if straight(hand):
        return 'straight', 4, max(number)
    
    if triples(hand):
        return 'triples', 3, max(number)
    
    if pairs(hand):
        # setup for pairings
        pairing = Counter(number).most_common(2)[0]
        possible_pairing = Counter(number).most_common(2)[1]
        
        # using Counter, determine the 2 most common repeating elements
        # if the 2 most common elements both repeat twice, then the hand is a 2-pair
        if pairing[1] == 2 and possible_pairing[1] == 2:
            return 'two pair', 2, max(pairing[0], possible_pairing[0])
        else:
            return 'pair', 1, pairing[0]
    
    # find the highest card if the hand does not have the above configurations
    highest_number = max(number)
    highest_index = number.index(highest_number)
    
    return 'high_card ' + hand[highest_index], 0, max(number)

In [946]:
print(poker_hand_ranking(hand1))
print(poker_hand_ranking(hand2))
print(poker_hand_ranking(hand3))
print(poker_hand_ranking(hand4))
print(poker_hand_ranking(hand5))
print(poker_hand_ranking(hand6))
print(poker_hand_ranking(hand7))

('royal flush', 9, 13)
('high_card Qs', 0, 12)
('four of a kind', 7, 10)
('straight', 4, 6)
('two pair', 2, 6)
('pair', 1, 3)
('two pair', 2, 5)


In [947]:
# answer key
hand1 = ["9s", "10s", "Js", "Ks", "Qs"] # straight flush, 10 to A for Hearts
hand2 = ["3h", "5h", "Qs", "9h", "Ad"] # highest card, Qs
hand3 = ["10s", "10c", "8d", "10d", "10h"] # 4 of a kind, 10
hand4 = ["6d", "2h", "4c", "5d", "3h"] # straight, 2 to 6
hand5 = ["6d", "6h", "4c", "5d", "5h"] # two pair, 6 & 5
hand6 = ["6d", "3d", "4c", "Ad", "3h"] # pair, 3
hand7 = ["2d", "4c", "2h", "5d", "5h"] # two pair, 2 & 5

------------
### **Stretch Content**

#### 2.  Implement a function `winner_is` that returns the winner given a dictionary with different players and their hands. 
**Example**

We define dictionary like
```python
round_1 = {"John" = ["10h", "Jh", "Qh", "Ah", "Kh"], 
        "Peter" = ["3h", "5h", "Qs", "9h", "Ad"]}
```

Our function returns the name of the winner:
> winner_is(round_1) --> "John"

One table can have up to 10 players.


In [948]:
def winner_is(round):
    # instantiate variables
    old_ranking = -1
    old_high_card = -1
    
    for player, hand in round.items():
        # determine hand ranking
        hand_config, ranking, high_card = poker_hand_ranking(hand)
        
        # compare this player's hand ranking to previous player's hand
        # if they are the same, then compare high cards
        if ranking > old_ranking or (ranking == old_ranking and high_card > old_high_card):
            winner = player
            winning_hand = hand
            winning_config = hand_config
            
            # replace old values
            old_ranking = ranking
            old_high_card = high_card
    
    return winner, winning_hand, winning_config

#### Tests

In [949]:
players = ['John', 'Mark', 'Christian', 'Emily', 'Pai', 'Elise', 'Seven']
hands = [hand1, hand2, hand3, hand4, hand5, hand6, hand7]
first_round = dict(zip(players, hands))

first_round

{'John': ['9s', '10s', 'Js', 'Ks', 'Qs'],
 'Mark': ['3h', '5h', 'Qs', '9h', 'Ad'],
 'Christian': ['10s', '10c', '8d', '10d', '10h'],
 'Emily': ['6d', '2h', '4c', '5d', '3h'],
 'Pai': ['6d', '6h', '4c', '5d', '5h'],
 'Elise': ['6d', '3d', '4c', 'Ad', '3h'],
 'Seven': ['2d', '4c', '2h', '5d', '5h']}

In [950]:
player, hand, config = winner_is(first_round)
print(player.upper() + " won with a " + config.upper() + ".")

JOHN won with a STRAIGHT FLUSH.


In [951]:
# answer key
# hand1 = ["10h", "Jh", "Qh", "Ah", "Kh"] # straight flush, 10 to A for Hearts
hand2 = ["3h", "5h", "Qs", "9h", "Ad"] # highest card, Qs
hand3 = ["10s", "10c", "8d", "10d", "10h"] # 4 of a kind, 10
hand4 = ["6d", "2h", "4c", "5d", "3h"] # straight, 2 to 6
hand5 = ["6d", "6h", "4c", "5d", "5h"] # two pair, 6 & 5
hand6 = ["6d", "3d", "4c", "Ad", "3h"] # pair, 3
hand7 = ["2d", "4c", "2h", "5d", "5h"] # two pair, 2 & 5

In [952]:
players = ['Mark', 'Christian', 'Emily', 'Pai', 'Elise', 'Seven']
hands = [hand2, hand3, hand4, hand5, hand6, hand7]
second_round = dict(zip(players, hands))

second_round

{'Mark': ['3h', '5h', 'Qs', '9h', 'Ad'],
 'Christian': ['10s', '10c', '8d', '10d', '10h'],
 'Emily': ['6d', '2h', '4c', '5d', '3h'],
 'Pai': ['6d', '6h', '4c', '5d', '5h'],
 'Elise': ['6d', '3d', '4c', 'Ad', '3h'],
 'Seven': ['2d', '4c', '2h', '5d', '5h']}

In [953]:
player, hand, config = winner_is(second_round)
print(player.upper() + " won with a " + config.upper() + ".")

CHRISTIAN won with a FOUR OF A KIND.


In [954]:
# answer key
# hand1 = ["10h", "Jh", "Qh", "Ah", "Kh"] # straight flush, 10 to A for Hearts
hand2 = ["3h", "5h", "Qs", "9h", "Ad"] # highest card, Qs
hand3 = ["10s", "10c", "8d", "10d", "10h"] # 4 of a kind, 10
hand4 = ["6d", "2h", "4c", "5d", "3h"] # straight, 2 to 6
hand5 = ["6d", "6h", "4c", "5d", "5h"] # two pair, 6 & 5
hand6 = ["6d", "3d", "4c", "Ad", "3h"] # pair, 3
hand7 = ["10h", "Jh", "Qh", "Kh", "Ah"] # two pair, 2 & 5

In [955]:
players = ['Mark', 'Christian', 'Emily', 'Pai', 'Elise', 'Seven']
hands = [hand2, hand3, hand4, hand5, hand6, hand7]
third_round = dict(zip(players, hands))

third_round

{'Mark': ['3h', '5h', 'Qs', '9h', 'Ad'],
 'Christian': ['10s', '10c', '8d', '10d', '10h'],
 'Emily': ['6d', '2h', '4c', '5d', '3h'],
 'Pai': ['6d', '6h', '4c', '5d', '5h'],
 'Elise': ['6d', '3d', '4c', 'Ad', '3h'],
 'Seven': ['10h', 'Jh', 'Qh', 'Kh', 'Ah']}

In [956]:
player, hand, config = winner_is(third_round)
print(player.upper() + " won with a " + config.upper() + ".")

SEVEN won with a ROYAL FLUSH.


#### 3. Create a function `distribute_cards` that randomly generates and gives 5 cards to every player given a list of player names.

**Example**

> distribute_cards(["John","Peter"])  -> round_1 = {"John" = ["10h", "Jh", "Qh", "Ah", "Kh"], 
        "Peter" = ["3h", "5h", "Qs", "9h", "Ad"]
}

In [957]:
def distribute_cards(players):
    # instantiate dictionary of hands
    hands = {}
    
    # define the suit conversion
    suits = [
        'h', # hearts
        's', # spades
        'd', # diamonds
        'c'] # clubs
    
    for i, player in enumerate(players):
        # reset list of cards (hand)
        hand = []
        
        # generate cards
        for _ in range(5):
            # reset card variable
            card = ""
            
            # generate random card with number and suit
            number = np.random.choice(face_key)
            suit = np.random.choice(suits)
            card = number + suit
            
            # evaluate if card already exists in this hand and other player's hands
            # if so, generate another card and reevaluate
            # if not, exit loop and add card to hand
            while card in (hand or hands):
                number = np.random.choice(face_key)
                suit = np.random.choice(suits)
                card = number + suit
                print(f"{card} already exists in {player}'s hand.")
                
            hand.append(card)
        
        # the resulting hand is added to list of hands and the
        # outer loop will generate another hand to evaluate
        hands[player] = hand
    
    return hands

In [958]:
players = [
    'John', 'Mark', 'Elise', 'Mary', 'June', 'April', 'Tuan', 'Sonny', 'Jerry', 'Julie', 'Sam']
distribute_cards(players)

Qh already exists in Mark's hand.
7h already exists in Mark's hand.
4s already exists in June's hand.
Jd already exists in Tuan's hand.


{'John': ['7c', 'Qh', 'As', '3s', '7s'],
 'Mark': ['6d', '9c', 'Qh', '7h', '3d'],
 'Elise': ['4c', '9h', 'Ad', '5s', '3s'],
 'Mary': ['3h', '8s', '2s', 'Ah', '9c'],
 'June': ['Kh', '4s', '2d', '6s', '4h'],
 'April': ['2h', '8h', '5d', 'Jh', '2c'],
 'Tuan': ['2d', '5c', '5s', 'Jd', 'Js'],
 'Sonny': ['Qc', '2c', '8d', 'Ah', '7d'],
 'Jerry': ['9s', '6d', 'As', '6s', 'Ad'],
 'Julie': ['Jc', '6d', '6c', 'Ah', '5s'],
 'Sam': ['Ad', '4h', 'Kd', 'Jd', 'Qd']}